## iPyLeaflet
[ipyleaflet](https://github.com/ellisonbg/ipyleaflet) is a bridge between jupyter notebooks and the [leaflet](http://leafletjs.com/)  javascript library for drawing maps.

ipyleaflet comes with a few examples notebooks (this notebook was derived from one) but very little documentation,
for more documentation read the [Leaflet IPA](http://leafletjs.com/reference.html)

For installation directions, see the README on [ipyleaflet](https://github.com/ellisonbg/ipyleaflet)

[GeoJson](https://tools.ietf.org/html/rfc7946#section-3.1.7) is a good way to add data on top of an ipyleaflet map.

For other backgrounds than open streets, you can alter the ipyleaflet.py file to point to one of [these map providers](https://leaflet-extras.github.io/leaflet-providers/preview/)

In [79]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

## Visualizing the distribution of the observations

## Load the required libraries

In [80]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
import pylab as plt

import sys
sys.path.append('./lib')

from leaflet import *

In [95]:
data_dir = "../../Data/Weather/"
from pickle import load
# with open(data_dir+'/PRCP_residuals_PCA.pickle','rb') as file:
#     Res_eig=load(file)
# print Res_eig.keys()

with open(data_dir+'/SNWD_mean.pickle','rb') as file:
    SNWD_mean=load(file)
print SNWD_mean.keys()

#stations.pkl
with open(data_dir+'/stations.pkl','rb') as file:
    Stations=load(file)
Stations.head()


['std', 'stations', 'mean']


,latitude,longitude,elevation,state,name,GSNFLAG,HCNFLAG,WMOID
ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
AF000040930,35.3170,69.0170,3366.0,NaN,NORTH-SALANG,GSN,NaN,40930.0
AG000060390,36.7167,3.2500,24.0,NaN,ALGER-DAR EL BEIDA,GSN,NaN,60390.0


In [102]:
import pandas as pd
# Eig=pd.DataFrame(Res_eig['eigen-vecs'].transpose())
# Eig.head()
Mean=pd.DataFrame(np.array([SNWD_mean['mean'],SNWD_mean['std']]).transpose())
Mean.head()

,0,1
0,5.472892,19.897081
1,13.908012,35.441143
2,122.586533,218.570819
3,45.748647,127.608368
4,41.955966,94.164433


In [103]:
# Eig['station']=Res_eig['stations']
# Eig=Eig.set_index('station')
# print Res_eig['stations'][:5]
# Eig.head()

Mean['station']=SNWD_mean['stations']
Mean=Mean.set_index('station')
print SNWD_mean['stations'][:5]
Mean.head()

[u'US1UTCH0002', u'US1UTDV0020', u'USC00100803', u'USC00103732', u'USC00105275']


,0,1
station,,
US1UTCH0002,5.472892,19.897081
US1UTDV0020,13.908012,35.441143
USC00100803,122.586533,218.570819
USC00103732,45.748647,127.608368
USC00105275,41.955966,94.164433


In [104]:
# table=Eig.join(Stations,how='left')
table = Mean.join(Stations,how='left')

In [105]:
table=table[['name','latitude','longitude','elevation',0,1]]
table.head(4)

,name,latitude,longitude,elevation,0,1
station,,,,,,
US1UTCH0002,HILLS 0.9 SW,41.6600,-111.8900,1371.6,5.472892,19.897081
US1UTDV0020,FARMINGTON 1.5 NNW,41.0078,-111.9067,1352.1,13.908012,35.441143
USC00100803,BERN,42.3353,-111.3850,1817.8,122.586533,218.570819
USC00103732,GRACE,42.5872,-111.7275,1691.6,45.748647,127.608368


In [88]:
import pylab as plt
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
print colors

[u'#1f77b4', u'#ff7f0e', u'#2ca02c', u'#d62728', u'#9467bd', u'#8c564b', u'#e377c2', u'#7f7f7f', u'#bcbd22', u'#17becf']


## Map

In [110]:
min_lat,max_lat,min_long,max_long = box = (40.88, 42.2458, -111.2417, -112.1817)
center = [(min_lat+max_lat)/2, (min_long+max_long)/2]
zoom = 9
m = Map(default_tiles=TileLayer(opacity=0.6), center=center, zoom=zoom)
r = Rectangle(bounds=[[min_lat,min_long],[max_lat,max_long]], weight=5, fill_opacity=0.0)
# m += r

# stations = [u'USC00420487', u'USS0011H25S', u'USC00428885', u'USC00428828', u'USS0011H55S', u'USC00424135', u'USS0011H57S', u'USW00024101', u'USC00427271', u'USW00094128', u'USC00422389', u'US1UTDV0020', u'USS0011H30S', u'USC00109698', u'USC00425082', u'USC00421601']
stations = [u'USC00105275', u'USS0011H25S', u'USC00428828', u'USC00425082', u'USS0011H55S', u'USC00424135', u'USC00109698', u'USC00421601', u'USS0011H30S', u'USW00024101', u'US1UTDV0020', u'USS0011H57S', u'USC00428885', u'USC00427271', u'USC00422389']

stations1 = [u'USC00420487', u'USS0011H25S', u'USC00428885', u'USC00428828', u'USS0011H55S', u'USC00424135', u'USS0011H57S'] 
stations2 = [u'USW00024101', u'USC00427271', u'USW00094128', u'USC00422389', u'US1UTDV0020', u'USS0011H30S'] 
stations3 = [u'USC00109698', u'USC00425082', u'USC00421601']
# stations4 = [u'US1UTDV0009', u'USC00424123', u'USC00423671', u'USC00422726']
lat_margin=(max_lat-min_lat)/4
long_margin=(max_long-min_long)/4
for index,row in table.iterrows():
        _lat=row['latitude']
        _long=row['longitude']
#     for col in range(4):
#         _coef=row[col]
#         if np.isnan(_coef):
#             continue
#         r=abs(_coef)/5
        mean = row[0]
        std = row[1]
        r = int(abs(std)*30)
#         color = colors[0]
        if(mean < 30):
            color = colors[0]
        elif(mean < 60):
            color = colors[1]
        else:
            color = colors[3]
#         color = colors[col]
        #print r
    #     r = 0.01
    #     if(index in stations):
    #         color = colors[1]
    #     if(index in stations1):
    #         color=colors[1]
    #     elif (index in stations2):
    #         color = colors[3]
    #     elif (index in stations3):
    #         color = colors[2]
    #     else:
    #     color=colors[0]
    #         color=colors[col]
#         signs=[[+1,+1],[+1,-1],[-1,-1],[-1,+1]]
#         lat_sign,long_sign=signs[col]
#         triangle=[(_lat,_long),(_lat+lat_sign*r,_long),(_lat,_long+long_sign*r),(_lat,_long)]
#         if _coef<0:
#             poly= Polygon(locations=triangle, weight=0,
#                     color=color, opacity=0, fill_opacity=0.7,
#                     fill_color=color)
#         else:
#             poly= Polygon(locations=triangle, weight=2,
#                     color=color, opacity=0.8, fill_opacity=0,
#                     fill_color=color)
        c = Circle(location=(_lat,_long),weight=3, fill_opacity=0.8, fill_color = color, color = color,radius=r)
        m += c
#         m +=poly   
m    

### excercises:
* Add a legend that relates the colors to values.
* Leaflet supports a variety of maps. See if you can get a topographical map as the background.

In [10]:
%matplotlib inline
pdf.plot.scatter(x='elevation',y='avg(coeff_1)');

NameError: name 'pdf' is not defined